In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
%config InlineBackend.figure_format='retina'

In [2]:
#export
from exp.nb_11 import *

# Serializing the model

In [3]:
imagenette_160

'https://s3.amazonaws.com/fast-ai-imageclas/imagenette-160'

In [4]:
#export
imagewoof_160 = 'https://s3.amazonaws.com/fast-ai-imageclas/imagewoof-160'

In [5]:
path = untar_data(imagewoof_160); path

PosixPath('/home/ubuntu/learnai/dl/data/imagewoof-160')

In [6]:
size = 128
bs = 64
tfms = [make_rgb, RandomResizedCrop(128, scale=(0.35, 1)), np_to_float, PilRandomFlip()]
val_tfms = [make_rgb, CenterCrop(size), np_to_float]

il = ImageList.from_files(path, tfms=tfms)
sd = SplitData.split_by_func(il, partial(grandparent_splitter, valid_name='val'))
ll = label_by_func(sd, parent_labeler)

ll.valid.x.tfms = val_tfms

data = ll.to_databunch(bs, c_in=3, c_out=10, num_workers=8)

In [7]:
len(il)

12954

In [8]:
loss_func = LabelSmoothingCrossEntropy()
opt_func = adam_opt(mom=0.9, mom_sqr=0.99, eps=1e-6, wd=1e-2)

In [9]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, norm=norm_imagenette)

In [14]:
def sched_1cycle(lr, pct_start=0.3, mom_start=0.95, mom_mid=0.85, mom_end=0.95):
    phases = create_phases(pct_start)
    sched_lr = combine_scheds(phases, cos_1cycle_anneal(lr/10., lr, lr/1e5))
    sched_mom = combine_scheds(phases, cos_1cycle_anneal(mom_start, mom_mid, mom_end))
    return [ParamScheduler('lr', sched_lr),
           ParamScheduler('mom', sched_mom)]

In [15]:
lr = 3e-3
pct_start = 0.5
cbsched = sched_1cycle(lr, pct_start)

In [16]:
learn.fit(40, cbsched)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,2.115075,0.254296,2.082309,0.256000,00:35
1,1.970873,0.334350,1.987623,0.286000,00:35
2,1.873440,0.385900,1.880044,0.358000,00:35
3,1.794246,0.418099,2.182671,0.344000,00:35
4,1.726671,0.459210,1.938796,0.384000,00:36
5,1.674430,0.482415,1.901978,0.380000,00:36
6,1.651203,0.496387,1.808902,0.422000,00:36
7,1.618130,0.516942,1.897751,0.370000,00:36
8,1.579994,0.533242,2.097953,0.362000,00:36
9,1.544113,0.547374,1.877809,0.394000,00:36


In [17]:
st = learn.model.state_dict()

In [18]:
type(st)

collections.OrderedDict

In [19]:
', '.join(st.keys())

'0.0.weight, 0.1.weight, 0.1.bias, 0.1.running_mean, 0.1.running_var, 0.1.num_batches_tracked, 1.0.weight, 1.1.weight, 1.1.bias, 1.1.running_mean, 1.1.running_var, 1.1.num_batches_tracked, 2.0.weight, 2.1.weight, 2.1.bias, 2.1.running_mean, 2.1.running_var, 2.1.num_batches_tracked, 4.0.convs.0.0.weight, 4.0.convs.0.1.weight, 4.0.convs.0.1.bias, 4.0.convs.0.1.running_mean, 4.0.convs.0.1.running_var, 4.0.convs.0.1.num_batches_tracked, 4.0.convs.1.0.weight, 4.0.convs.1.1.weight, 4.0.convs.1.1.bias, 4.0.convs.1.1.running_mean, 4.0.convs.1.1.running_var, 4.0.convs.1.1.num_batches_tracked, 5.0.convs.0.0.weight, 5.0.convs.0.1.weight, 5.0.convs.0.1.bias, 5.0.convs.0.1.running_mean, 5.0.convs.0.1.running_var, 5.0.convs.0.1.num_batches_tracked, 5.0.convs.1.0.weight, 5.0.convs.1.1.weight, 5.0.convs.1.1.bias, 5.0.convs.1.1.running_mean, 5.0.convs.1.1.running_var, 5.0.convs.1.1.num_batches_tracked, 5.0.idconv.0.weight, 5.0.idconv.1.weight, 5.0.idconv.1.bias, 5.0.idconv.1.running_mean, 5.0.idconv.1.

In [20]:
st['10.bias']

tensor([ 0.0096,  0.0211, -0.0546,  0.0071,  0.0194,  0.0011,  0.0198, -0.0215,
         0.0013, -0.0111], device='cuda:0')

In [21]:
mdl_path = path/'models'
mdl_path.mkdir(exist_ok=True)

In [22]:
torch.save(st, mdl_path/'iw5')

## Pets

In [23]:
#export
pets_url = 'https://s3.amazonaws.com/fast-ai-imageclas/oxford-iiit-pet'

In [24]:
pets = untar_data(pets_url)

In [25]:
pets_path = pets/'images'

In [26]:
il = ImageList.from_files(pets_path, tfms=tfms)

In [27]:
il

ImageList(7390 items)
[PosixPath('/home/ubuntu/learnai/dl/data/oxford-iiit-pet/images/keeshond_34.jpg'), PosixPath('/home/ubuntu/learnai/dl/data/oxford-iiit-pet/images/Siamese_178.jpg'), PosixPath('/home/ubuntu/learnai/dl/data/oxford-iiit-pet/images/german_shorthaired_94.jpg'), PosixPath('/home/ubuntu/learnai/dl/data/oxford-iiit-pet/images/Abyssinian_92.jpg'), PosixPath('/home/ubuntu/learnai/dl/data/oxford-iiit-pet/images/basset_hound_111.jpg'), PosixPath('/home/ubuntu/learnai/dl/data/oxford-iiit-pet/images/Russian_Blue_194.jpg'), PosixPath('/home/ubuntu/learnai/dl/data/oxford-iiit-pet/images/staffordshire_bull_terrier_91.jpg'), PosixPath('/home/ubuntu/learnai/dl/data/oxford-iiit-pet/images/Persian_69.jpg'), PosixPath('/home/ubuntu/learnai/dl/data/oxford-iiit-pet/images/english_setter_33.jpg'), PosixPath('/home/ubuntu/learnai/dl/data/oxford-iiit-pet/images/Russian_Blue_155.jpg')...]
Path: /home/ubuntu/learnai/dl/data/oxford-iiit-pet/images

In [28]:
#export
def random_splitter(fn, p_valid):
    return random.random() < p_valid

In [29]:
random.seed(42)

In [30]:
sd = SplitData.split_by_func(il, partial(random_splitter, p_valid=0.1))

In [31]:
sd

SplitData
Train: ImageList(6667 items)
[PosixPath('/home/ubuntu/learnai/dl/data/oxford-iiit-pet/images/keeshond_34.jpg'), PosixPath('/home/ubuntu/learnai/dl/data/oxford-iiit-pet/images/german_shorthaired_94.jpg'), PosixPath('/home/ubuntu/learnai/dl/data/oxford-iiit-pet/images/Abyssinian_92.jpg'), PosixPath('/home/ubuntu/learnai/dl/data/oxford-iiit-pet/images/basset_hound_111.jpg'), PosixPath('/home/ubuntu/learnai/dl/data/oxford-iiit-pet/images/Russian_Blue_194.jpg'), PosixPath('/home/ubuntu/learnai/dl/data/oxford-iiit-pet/images/staffordshire_bull_terrier_91.jpg'), PosixPath('/home/ubuntu/learnai/dl/data/oxford-iiit-pet/images/english_setter_33.jpg'), PosixPath('/home/ubuntu/learnai/dl/data/oxford-iiit-pet/images/Bengal_62.jpg'), PosixPath('/home/ubuntu/learnai/dl/data/oxford-iiit-pet/images/yorkshire_terrier_136.jpg'), PosixPath('/home/ubuntu/learnai/dl/data/oxford-iiit-pet/images/beagle_82.jpg')...]
Path: /home/ubuntu/learnai/dl/data/oxford-iiit-pet/images
Valid: ImageList(723 items)

In [32]:
n = il.items[0].name; n

'keeshond_34.jpg'

In [33]:
re.findall(r'^(.*)_\d+.jpg$', n)[0]

'keeshond'

In [34]:
def pet_labeler(fn):
    return re.findall(r'^(.*)_\d+.jpg$', fn.name)[0]

In [51]:
#export
class CategoryProcessor(Processor):
    def __init__(self): self.vocab=None

    def __call__(self, items):
        #The vocab is defined on the first use.
        if self.vocab is None:
            self.vocab = uniqueify(items)
            self.otoi  = {v:k for k,v in enumerate(self.vocab)}
        return [self.proc1(o) for o in items]
    def proc1(self, item):  return self.otoi[item]

    def deprocess(self, idxs):
        assert self.vocab is not None
        return [self.deproc1(idx) for idx in idxs]
    def deproc1(self, idx): return self.vocab[idx]

In [52]:
proc = CategoryProcessor()

In [53]:
def _label_by_func(ds, f, cls=ItemList):
    return cls([f(o) for o in ds.items], path=ds.path)

In [54]:
class LabeledData():
    def process(self, il, proc): return il.new(compose(il.items, proc))

    def __init__(self, x, y, proc_x=None, proc_y=None):
        self.x,self.y = self.process(x, proc_x),self.process(y, proc_y)
        self.proc_x,self.proc_y = proc_x,proc_y

    def __repr__(self): return f'{self.__class__.__name__}\nx: {self.x}\ny: {self.y}\n'
    def __getitem__(self,idx): return self.x[idx],self.y[idx]
    def __len__(self): return len(self.x)

    def x_obj(self, idx): return self.obj(self.x, idx, self.proc_x)
    def y_obj(self, idx): return self.obj(self.y, idx, self.proc_y)

    def obj(self, items, idx, procs):
        isint = isinstance(idx, int) or (isinstance(idx,torch.LongTensor) and not idx.ndim)
        item = items[idx]
        for proc in reversed(listify(procs)):
            item = proc.deproc1(item) if isint else proc.deprocess(item)
        return item

    @classmethod
    def label_by_func(cls, il, f, proc_x=None, proc_y=None):
        return cls(il, _label_by_func(il, f), proc_x=proc_x, proc_y=proc_y)

In [55]:
def label_by_func(sd, f, proc_x=None, proc_y=None):
    train = LabeledData.label_by_func(sd.train, f, proc_x=proc_x, proc_y=proc_y)
    valid = LabeledData.label_by_func(sd.valid, f, proc_x=proc_x, proc_y=proc_y)
    return SplitData(train,valid)

In [56]:
ll = label_by_func(sd, pet_labeler, proc_y=proc)

In [57]:
', '.join(proc.vocab)

'keeshond, german_shorthaired, Abyssinian, basset_hound, Russian_Blue, staffordshire_bull_terrier, english_setter, Bengal, yorkshire_terrier, beagle, newfoundland, Birman, american_bulldog, japanese_chin, British_Shorthair, Ragdoll, leonberger, Maine_Coon, miniature_pinscher, pomeranian, english_cocker_spaniel, pug, Persian, shiba_inu, wheaten_terrier, great_pyrenees, saint_bernard, Egyptian_Mau, havanese, chihuahua, Bombay, american_pit_bull_terrier, Sphynx, scottish_terrier, boxer, Siamese, samoyed'

In [58]:
ll.valid.x.tfms = val_tfms

In [59]:
c_out = len(proc.vocab)

In [60]:
data = ll.to_databunch(bs, c_in=3, c_out=c_out, num_workers=8)

In [61]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, norm=norm_imagenette)

In [62]:
learn.fit(5, cbsched)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,3.471157,0.086246,3.450891,0.096819,00:22
1,3.306132,0.134243,3.399893,0.116183,00:22
2,3.083709,0.191390,3.320758,0.149378,00:22
3,2.806000,0.273136,2.686408,0.312586,00:22
4,2.546781,0.362982,2.557566,0.370678,00:22


## Custom head

In [65]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, c_out=10, norm=norm_imagenette)

In [66]:
st = torch.load(mdl_path/'iw5')

In [67]:
m = learn.model

In [68]:
m.load_state_dict(st)

<All keys matched successfully>

In [69]:
cut = next(i for i,o in enumerate(m.children()) if isinstance(o, nn.AdaptiveAvgPool2d))
m_cut = m[:cut]

In [70]:
xb, yb = get_batch(data.valid_dl, learn)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time


In [71]:
pred = m_cut(xb)

In [72]:
pred.shape

torch.Size([128, 512, 4, 4])

In [73]:
ni = pred.shape[1]

In [74]:
#export
class AdaptiveConcatPool2d(nn.Module):
    def __init__(self, sz=1):
        super().__init__()
        self.output_sz = sz
        self.ap = nn.AdaptiveAvgPool2d(sz)
        self.mp = nn.AdaptiveAvgPool2d(sz)
    
    def forward(self, x):
        return torch.cat([self.mp(x), self.ap(x)], 1)

In [75]:
nh = 40

m_new = nn.Sequential(m_cut, AdaptiveConcatPool2d(), Flatten(), nn.Linear(ni*2, data.c_out))

In [76]:
learn.model = m_new

In [77]:
learn.fit(5, cbsched)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,3.091890,0.241338,2.483180,0.365145,00:22
1,2.235458,0.456277,2.618409,0.383126,00:22
2,2.006195,0.540123,2.116077,0.499308,00:22
3,1.797752,0.623069,1.795615,0.621024,00:22
4,1.617882,0.699865,1.681460,0.686030,00:22


In [78]:
learn.fit(5, cbsched)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,1.582237,0.713064,1.691652,0.654219,00:22
1,1.648227,0.681416,2.280815,0.443983,00:22
2,1.691083,0.659967,1.978232,0.558783,00:22
3,1.567732,0.707065,1.726649,0.645920,00:22
4,1.418059,0.780861,1.543990,0.721992,00:22


## adapt_model and gradual unfreezing

In [79]:
def adapt_model(learn, data):
    cut = next(i for i,o in enumerate(learn.model.children()) if isinstance(o, nn.AdaptiveAvgPool2d))
    m_cut = learn.model[:cut]
    xb, yb = get_batch(data.valid_dl, learn)
    pred = m_cut(xb)
    ni = pred.shape[1]
    m_new = nn.Sequential(m_cut, AdaptiveConcatPool2d(), Flatten(),
                         nn.Linear(ni*2, data.c_out))
    learn.model = m_new

In [80]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, c_out=10, norm=norm_imagenette)
learn.model.load_state_dict(torch.load(mdl_path/'iw5'))

<All keys matched successfully>

In [81]:
adapt_model(learn, data)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time


In [82]:
for p in learn.model[0].parameters():
    p.requires_grad_(False)

In [83]:
learn.fit(3, sched_1cycle(1e-2, 0.5))

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,2.878998,0.277636,2.424610,0.388658,00:18
1,2.398667,0.411279,2.270540,0.445367,00:18
2,2.163579,0.493775,2.113665,0.522822,00:18


In [84]:
for p in learn.model[0].parameters():
    p.requires_grad_(True)

In [86]:
learn.fit(5, cbsched, reset_opt=True)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,1.517543,0.746213,1.723782,0.662517,00:22
1,1.572683,0.719664,1.819969,0.601660,00:22
2,1.628756,0.688766,1.905382,0.590595,00:22
3,1.506087,0.748013,1.647199,0.672199,00:23
4,1.374243,0.801710,1.533309,0.734440,00:22


## Batch norm transfer

In [88]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, c_out=10, norm=norm_imagenette)
learn.model.load_state_dict(torch.load(mdl_path/'iw5'))
adapt_model(learn, data)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time


In [92]:
def apply_mod(m, f):
    f(m)
    for l in m.children():
        apply_mod(l, f)
        
def set_grad(m, b):
    if isinstance(m, (nn.Linear, nn.BatchNorm2d)):
        return
    if hasattr(m, 'weight'):
        for p in m.parameters():
            p.requires_grad_(b)

In [90]:
learn.fit(6, sched_1cycle(1e-2, 0.5))

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,2.730455,0.319484,2.816823,0.268326,00:22
1,2.218069,0.466027,2.809505,0.300138,00:22
2,2.184974,0.472476,3.699703,0.165975,00:22
3,2.050885,0.529923,2.815511,0.345781,00:22
4,1.760509,0.636568,1.788362,0.641770,00:23
5,1.539597,0.730913,1.631113,0.680498,00:22


In [93]:
apply_mod(learn.model, partial(set_grad, b=True))

In [94]:
learn.fit(10, cbsched, reset_opt=True)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,1.496431,0.747113,1.594805,0.704011,00:22
1,1.467221,0.761362,1.643050,0.672199,00:22
2,1.472337,0.756262,1.654656,0.697095,00:22
3,1.483606,0.751612,1.737057,0.652835,00:22
4,1.475295,0.745763,1.658199,0.662517,00:22
5,1.451684,0.760012,1.659700,0.668050,00:22
6,1.390008,0.783711,1.573680,0.698479,00:22
7,1.321054,0.815359,1.462602,0.735823,00:22
8,1.257006,0.844458,1.445695,0.752420,00:22
9,1.222524,0.859457,1.449738,0.745505,00:22


PyTorch already has an `apply` method that we can use:

In [ ]:
learn.model.apply(partial(set_grad, b=False))

## Discriminative LR and param groups

In [95]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, c_out=10, norm=norm_imagenette)

In [96]:
learn.model.load_state_dict(torch.load(mdl_path/'iw5'))
adapt_model(learn, data)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time


In [98]:
def bn_splitter(m):
    def _bn_splitter(l, g1, g2):
        if isinstance(l, nn.BatchNorm2d):
            g2 += l.parameters()
        elif hasattr(l, 'weight'):
            g1 += l.parameters()
        for ll in l.children():
            _bn_splitter(ll, g1, g2)
            
    g1, g2 = [], []
    _bn_splitter(m[0], g1, g2)
    
    g2 += m[1:].parameters()
    return g1, g2

In [99]:
a, b = bn_splitter(learn.model)

In [102]:
test_eq(len(a) + len(b), len(list(m.parameters())))

In [103]:
Learner.ALL_CBS

{'after_backward',
 'after_batch',
 'after_cancel_batch',
 'after_cancel_epoch',
 'after_cancel_train',
 'after_epoch',
 'after_fit',
 'after_loss',
 'after_pred',
 'after_step',
 'begin_batch',
 'begin_epoch',
 'begin_fit',
 'begin_validate'}

In [104]:
#export
from types import SimpleNamespace
cb_types = SimpleNamespace(**{o:o for o in Learner.ALL_CBS})

In [105]:
cb_types.after_backward

'after_backward'

In [106]:
#export
class DebugCallback(Callback):
    _order = 999
    def __init__(self, cb_name, f=None):
        self.cb_name, self.f = cb_name, f
        
    def __call__(self, cb_name):
        if cb_name == self.cb_name:
            if self.f:
                self.f(self.run)
            else:
                set_trace()

In [107]:
#export
def sched_1cycle(lrs, pct_start=0.3, mom_start=0.95, mom_mid=0.85, mom_end=0.95):
    phases = create_phases(pct_start)
    sched_lr = [combine_scheds(phases, cos_1cycle_anneal(lr/10., lr, lr/1e5)) for lr in lrs]
    sched_mom = combine_scheds(phases, cos_1cycle_anneal(mom_start, mom_mid, mom_end))
    return [ParamScheduler('lr', sched_lr),
           ParamScheduler('mom', sched_mom)]

In [108]:
disc_lr_sched = sched_1cycle([0,3e-2], 0.5)

In [110]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, c_out=10, norm=norm_imagenette, splitter=bn_splitter)
learn.model.load_state_dict(torch.load(mdl_path/'iw5'))
adapt_model(learn, data)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time


In [111]:
def _print_det(o):
    print(len(o.opt.param_groups), o.opt.hypers)
    raise CancelTrainException()
    
learn.fit(1, disc_lr_sched + [DebugCallback(cb_types.after_batch, _print_det)])

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time


2 [{'mom': 0.9499999999999997, 'mom_sqr': 0.99, 'eps': 1e-06, 'wd': 0.01, 'lr': 0.0, 'sqr_mom': 0.99}, {'mom': 0.9499999999999997, 'mom_sqr': 0.99, 'eps': 1e-06, 'wd': 0.01, 'lr': 0.0030000000000000512, 'sqr_mom': 0.99}]


In [112]:
learn.fit(3, disc_lr_sched)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,2.642950,0.332383,2.427750,0.420470,00:22
1,2.293073,0.433778,2.259629,0.455048,00:22
2,2.055979,0.533973,1.956673,0.564315,00:22


In [113]:
disc_lr_sched = sched_1cycle([1e-3, 1e-2], 0.3)

In [114]:
learn.fit(5, disc_lr_sched)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,1.899286,0.593670,1.985661,0.564315,00:22
1,1.920151,0.573721,2.016438,0.538036,00:22
2,1.804744,0.624269,1.889889,0.573997,00:22
3,1.695110,0.667767,1.731124,0.654219,00:22
4,1.592891,0.711414,1.725438,0.656985,00:22


# Export

In [115]:
!python notebook2script.py 11a_transfer_learning.ipynb

converted 11a_transfer_learning.ipynb to nb_11a.py
